This is the Test Lab of Calling LLM Using Google API or Gemini. 

This cell:

Loads your environment, connects to your LLM engine (OpenAI), brings in a custom website scraper, and prepares pretty output rendering.

In [1]:
# imports

import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI  # same client, but we’ll point it to Gemini’s endpoint



In [2]:
load_dotenv(override=True)
api_key = os.getenv('GEMINI_API_KEY') or os.getenv('GOOGLE_API_KEY')

if not api_key:
    print("No Gemini API key found. Please create one at https://aistudio.google.com/app/apikey.")
elif not api_key.startswith("AI"):
    print("A key was found, but it doesn’t look like a valid Gemini key (should start with 'AI').")
elif api_key.strip() != api_key:
    print("Your Gemini key has extra spaces — remove them in your .env file.")
else:
    print("Gemini API key found and looks good ✅")


Gemini API key found and looks good ✅


# Let's make a quick call to a Frontier model to get started, as a preview!

In [3]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, Gemini This is my first ever message to you! Hi!"

messages = [{"role": "user", "content": message}]

messages


[{'role': 'user',
  'content': 'Hello, Gemini This is my first ever message to you! Hi!'}]

In [8]:
gemini_key = os.getenv("GEMINI_API_KEY") or os.getenv("GOOGLE_API_KEY")

# Create client pointed at Gemini's OpenAI-compatible endpoint
gemini = OpenAI(
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=gemini_key
)

# Send chat message
response = gemini.chat.completions.create(
    model="gemini-2.0-flash",          # or gemini-2.0-flash, etc.
    messages=messages
)

# Display Gemini's reply
print(response.choices[0].message.content)

Hello! It's so nice to receive your first message! Welcome! I'm happy to be here. Is there anything I can help you with today?



In [9]:
# Let's try out this utility

ed = fetch_website_contents("https://edwarddonner.com")
print(ed)

Home - Edward Donner

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Conne

## Types of prompts

Models like GPT have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [33]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a news reporter like a reporter of BBC or CNN.You read the news and report the same to the user.in a detailed userfriendly manner.
Respond in Markdown, without wrapping your response in code blocks.
"""

In [34]:
# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

## And now let's build useful messages for GPT-4.1-mini, using a function

In [35]:
# See how this function creates exactly the format above

def messages_for(website_content: str):
    """Return a properly formatted message list for the LLM."""
    return [
        {"role": "system", "content": system_prompt.strip()},
        {"role": "user", "content": f"{user_prompt_prefix.strip()}\n{website_content.strip()}"}
    ]


In [36]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are a news reporter like a reporter of BBC or CNN.You read the news and report the same to the user.in a detailed userfriendly manner.\nRespond in Markdown, without wrapping your response in code blocks.'},
 {'role': 'user',
  'content': 'Here are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\nHome - Edward Donner\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, pos

## Time to bring it together - the API for OpenAI is very simple!

In [37]:
def summarize(url):
    website = fetch_website_contents(url)
    response = gemini.chat.completions.create(
        model = "gemini-2.0-flash-lite",  # works with the OpenAI-compatible endpoint
        messages = messages_for(website)
    )
    return response.choices[0].message.content


In [31]:
summarize("https://edwarddonner.com")

'So, this is Ed\'s corner of the internet. Apparently, he\'s a code-slinging, LLM-loving DJ-wannabe who\'s also been a startup founder (twice!). He\'s got a gig at Nebula.io, where they\'re using AI to, I guess, help people not waste their lives. He also has a cool project called "Outsmart" where AI models fight each other, which sounds way more interesting than my job.\n\nHe\'s also got some old blog posts about AI, including one from 2025. Either he\'s from the future or time travel is the next big thing, which would actually explain a lot.\n'

In [38]:
# A function to display this nicely in the output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [39]:
display_summary("https://www.indiatoday.in")

Hello there! I'm here to give you a quick rundown of what's happening on the India Today website.

**Overall Summary:**

India Today is a comprehensive news and media platform. It offers a wide range of content, including news, live TV, magazines, and various sections covering topics like India, global affairs, business, sports, technology, entertainment, and lifestyle. It also has a strong presence on social media and provides options for users to download their app.

**Latest News Highlights:**

*   **Al-Falah University Under Scrutiny:** Following the Red Fort blast, Al-Falah University in Haryana is facing difficulties. Its accreditation and funding are under investigation. A doctor involved in the blast worked at the university.
*   **Delhi Pollution:** The Supreme Court has expressed serious concerns about Delhi's pollution levels, stating that even masks might not be sufficient protection.


# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [21]:
display_summary("https://cnn.com")

Ah, CNN, where you can catch up on the latest wars, Trump's shenanigans, and whether or not your avocado toast is contributing to climate change. Plus, they're super interested in your ad feedback, because who isn't thrilled about pop-up surveys?


In [ ]:
display_summary("https://anthropic.com")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [ ]:
# Step 1: Create your prompts

system_prompt = "something here"
user_prompt = """
    Lots of text
    Can be pasted here
"""

# Step 2: Make the messages list

messages = [] # fill this in

# Step 3: Call OpenAI
# response =

# Step 4: print the result
# print(

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

Here are good instructions courtesy of an AI friend:  
https://chatgpt.com/share/677a9cb5-c64c-8012-99e0-e06e88afd293